In [5]:
import pandas as pd
import numpy as np

In [26]:
data = pd.read_csv("./Dataset/household_power_consumption.txt", sep=";")

C:\Users\nitis\AppData\Local\Temp\ipykernel_15404\1837839935.py:1: DtypeWarning: Columns (2,3,4,5,6,7) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv("./Dataset/household_power_consumption.txt", sep=";")


In [27]:
data["datetime"] = data["Date"] + " " + data["Time"]
data["datetime"] = pd.to_datetime(data["datetime"], dayfirst=True)

In [28]:
data.drop(columns=["Date", "Time"], inplace=True)
data.set_index("datetime", inplace=True)


In [30]:
for col in data.columns:
    data[col].replace("?", np.nan, inplace=True)
    data[col] = data[col].astype(np.float32)

data.fillna(method='ffill', axis=0, inplace=True)

C:\Users\nitis\AppData\Local\Temp\ipykernel_15404\1964392206.py:5: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  data.fillna(method='ffill', axis=0, inplace=True)


In [31]:
data_resampled = data.resample("H").sum()


,Global_active_power,Global_reactive_power,Voltage,Global_intensity,Sub_metering_1,Sub_metering_2,Sub_metering_3
datetime,,,,,,,
2006-12-16 17:24:00,4.216,0.418,234.839996,18.4,0.0,1.0,17.0
2006-12-16 17:25:00,5.360,0.436,233.630005,23.0,0.0,1.0,16.0
2006-12-16 17:26:00,5.374,0.498,233.289993,23.0,0.0,2.0,17.0
2006-12-16 17:27:00,5.388,0.502,233.740005,23.0,0.0,1.0,17.0
2006-12-16 17:28:00,3.666,0.528,235.679993,15.8,0.0,1.0,17.0


In [32]:
data_resampled.head()

,Global_active_power,Global_reactive_power,Voltage,Global_intensity,Sub_metering_1,Sub_metering_2,Sub_metering_3
datetime,,,,,,,
2006-12-16 17:00:00,152.024002,8.243999,8447.179688,651.599976,0.0,19.0,607.0
2006-12-16 18:00:00,217.932007,4.802000,14074.809570,936.000000,0.0,403.0,1012.0
2006-12-16 19:00:00,204.013992,5.114000,13993.950195,870.200012,0.0,86.0,1001.0
2006-12-16 20:00:00,196.113998,4.506000,14044.290039,835.000000,0.0,0.0,1007.0
2006-12-16 21:00:00,183.388000,4.600000,14229.519531,782.799988,0.0,25.0,1033.0
